In [16]:
import re
import sys
import pandas as pd
import numpy as np


from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download("stopwords")

import seaborn as sns

from collections import Counter

from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.model_selection import GridSearchCV


[nltk_data] Downloading package punkt to /home/lipsa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/lipsa/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/lipsa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
train_data = pd.read_csv("Desktop/AI Docs/project/train.csv", encoding='ISO-8859-1')
#train_data.isnull()
#train_data.dropna(inplace = True)
train_data.head()

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...


In [ ]:
test_data = pd.read_csv("Desktop/AI Docs/project/test.csv", encoding='ISO-8859-1')
test_data.head()

In [ ]:
test_data.info()

    Data Visualization: 

In [ ]:
# #Pre-processing (Data Cleaning) :
# def preprocessed(tweets):
    
#     #Convert to lowercase:
#     tweets.lower()
#     tweets.lstrip()
#     tweets.rstrip()
#     #Remove punctuations:
#     tweets = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), '', tweets)
    
#     # Remove repetative characters:
#     tweets = re.sub(r'(.)\1+', r'\1\1', tweets)
    
#     #Remove urls and replace with " "
#     tweets = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' URL ', tweets)
    
#     #Remove @username and replace with " "
#     tweets = re.sub('@[^\s]+','USER_MENTION', tweets)
    
#     #Remove multiple white spaces and replace with a single 
#     tweets = re.sub('[\s]+',' ',tweets)
   
#     #Remove "Hashtag(#word)" and replace with  "word"
#     tweets = re.sub(r'#([^\s]+)', r'\1', tweets)
    
#     #Remove 2+ dots and replace with space
#     tweets = re.sub(r'\.{2,}', ' ', tweets)
    
#     #Remove emojis with either EMO_POS or EMO_NEG
#     #tweets = emoji_type(tweets)
    
#     #Remove special characters except letters and numbers and replace with space
#     tweets = re.sub('[^ a-zA-Z0-9]', ' ', tweets)
    
#     #Remove numbers with space [Numbers doesn't help to determine positive or negative emotions]
#     tweets = re.sub('[0-9]', ' ', tweets)
    
#     #Remove retweets and CC [Retweets always begins with RT]
#     tweets = re.sub('RT|cc', '', tweets)
    
#     return tweets
    
                       
                     

In [ ]:
sns.countplot(x='Sentiment',data=train_data)

In [20]:
#Check the different Emotions present in the tweets:
count_emoji = []
tweets = train_data.SentimentText.str.cat()
emojis = set(re.findall(r" ([:;Xx][-']?.) ",tweets))

for e in emojis:
    count_emoji.append((tweets.count(e), e))
sorted(count_emoji,reverse=True)

[(3281, ':/'),
 (2874, 'x '),
 (2626, ': '),
 (1339, 'x@'),
 (1214, 'xx'),
 (1162, 'xa'),
 (984, ';3'),
 (887, 'xp'),
 (842, 'xo'),
 (713, ';)'),
 (483, 'xe'),
 (431, ';I'),
 (353, ';.'),
 (254, 'xD'),
 (251, 'x.'),
 (245, '::'),
 (234, 'X '),
 (217, ';t'),
 (209, ';s'),
 (185, ':O'),
 (176, ':3'),
 (166, ';D'),
 (159, ":'"),
 (157, 'XD'),
 (146, 'x3'),
 (142, ':p'),
 (126, ":'("),
 (118, ':@'),
 (117, 'xh'),
 (117, ':S'),
 (109, 'xm'),
 (104, ';p'),
 (104, ';-)'),
 (92, ':|'),
 (91, 'x,'),
 (89, ';P'),
 (76, 'xd'),
 (75, ';o'),
 (75, ';d'),
 (71, ':o'),
 (65, 'XX'),
 (63, ':L'),
 (59, 'Xx'),
 (59, ':1'),
 (58, ':]'),
 (57, ':s'),
 (56, ':0'),
 (54, 'XO'),
 (44, ';;'),
 (43, ';('),
 (38, ':-D'),
 (37, 'xk'),
 (36, 'XT'),
 (35, 'x?'),
 (35, 'x)'),
 (34, 'x2'),
 (33, ';/'),
 (32, 'x:'),
 (32, ':\\'),
 (31, 'x-'),
 (27, 'Xo'),
 (27, 'XP'),
 (27, ':-/'),
 (26, ':-P'),
 (25, ':*'),
 (23, 'xX'),
 (22, ":')"),
 (17, 'xP'),
 (16, ':['),
 (16, ':-p'),
 (14, 'x]'),
 (14, 'XM'),
 (13, ':-O'),
 (1

In [22]:
#tweets = train_data.SentimentText.str.cat()    
POSITIVE_EMO = r" ([xX;:]-?[dD)]|:-?[\)]|[;:][pP]) "
NEGATIVE_EMO = r" (:'?[/|\(]) "
print("POSITIVE emoticons:", set(re.findall(POSITIVE_EMO, tweets)))
print("NEGATIVE emoticons:", set(re.findall(NEGATIVE_EMO, tweets)))

POSITIVE emoticons: {';D', ';-D', ':D', 'x)', ';d', ':p', ';P', ';p', 'xD', 'XD', ':-D', ';-)', ':d', 'xd', ';)'}
NEGATIVE emoticons: {':|', ':/', ':(', ":'("}


In [ ]:
#train_data['SentimentText'] = train_data['SentimentText'].apply(preprocessed)

Apply Stemming

In [ ]:
# ps = PorterStemmer()

# def tokenise_tweets(text):
#     if not text:
#         print('The text to be tokenized is a None type. Defaulting to blank string.')
#         te`xt = ''
#     return word_tokenize(text)

# def stemming_tweets(words):
#         #ps = PorterStemmer() 
#        # words = word_tokenize(tweets) 
#         stemmed_words = []
#         for w in words:
#             w = ps.stem(w)
#             stemmed_words.append(w)
#         return stemmed_words
#             #print(w, " : ", str(ps.stem(w))) 

# train_data['text'] = train_data['SentimentText'].apply(tokenise_tweets)
# train_data['StemmedText'] = train_data['text'].apply(stemming_tweets)
# train_data.head()

In [ ]:
# Tally occurrences of words in a list
# cnt = Counter()
# for word in train_data.index:
#     cnt.update(train_data.loc[word,"text"])
# cnt.most_common(10)

In [ ]:
# def most_occured_words(text):
#     words = word_tokenize(text)
#     freq_dist = nltk.FreqDist(words)
#     print("There is %d different words" % len(set(words)))
#     return sorted(freq_dist,key=freq_dist.__getitem__, reverse=True)

# most_occured_words(train_data.SentimentText.str.cat())

STOP WORDS

In [23]:
def most_used_words(text):
    tokens = word_tokenize(text)
    frequency_dist = nltk.FreqDist(tokens)
    print("There is %d different words" % len(set(tokens)))
    return sorted(frequency_dist,key=frequency_dist.__getitem__, reverse=True)

most_used_words(train_data.SentimentText.str.cat())[:100]

There is 133899 different words


['@',
 '!',
 '.',
 'I',
 ',',
 'to',
 'the',
 'you',
 '?',
 'a',
 'it',
 'i',
 '...',
 ';',
 'and',
 '&',
 'my',
 'for',
 'is',
 'that',
 "'s",
 "n't",
 'in',
 'of',
 'me',
 'have',
 'on',
 'quot',
 "'m",
 'so',
 ':',
 'but',
 '#',
 'do',
 'was',
 'be',
 'not',
 'your',
 'are',
 'just',
 'with',
 'like',
 '-',
 'at',
 'too',
 'get',
 'good',
 'u',
 'up',
 'know',
 'all',
 'this',
 'now',
 'no',
 'we',
 'out',
 ')',
 'love',
 'can',
 '(',
 'what',
 'one',
 'will',
 'lol',
 'go',
 'about',
 'did',
 "'ll",
 'got',
 'amp',
 'there',
 'day',
 'http',
 'see',
 "'re",
 'if',
 'time',
 'they',
 'think',
 'as',
 'when',
 'from',
 'You',
 'It',
 'going',
 'really',
 'am',
 'work',
 'well',
 'had',
 'would',
 'how',
 'he',
 'here',
 'some',
 'thanks',
 'back',
 'im',
 'haha',
 'or']

In [24]:
mw = most_used_words(train_data.SentimentText.str.cat())
most_words = []
for w in mw:
    if len(most_words) == 1000:
        break
    if w in stopwords.words("english"):
        continue
    else:
        most_words.append(w)
        
sorted(most_words)

There is 133899 different words


['!',
 '#',
 '$',
 '%',
 '&',
 "'",
 "'d",
 "'ll",
 "'m",
 "'re",
 "'s",
 "'ve",
 '(',
 ')',
 '*',
 '*hugs*',
 '*sigh*',
 '+',
 ',',
 '-',
 '--',
 '.',
 '..',
 '...',
 '/',
 '1',
 '10',
 '100',
 '12',
 '1st',
 '2',
 '20',
 '2nd',
 '3',
 '30',
 '30SECONDSTOMARS',
 '4',
 '5',
 '6',
 '7',
 '8',
 ':',
 ';',
 '=',
 '?',
 '@',
 'A',
 'AND',
 'Ah',
 'AlexAllTimeLow',
 'All',
 'Also',
 'Alyssa_Milano',
 'Am',
 'And',
 'Are',
 'As',
 'At',
 'Aw',
 'Awesome',
 'Aww',
 'Awww',
 'BSB',
 'Birthday',
 'But',
 'Ca',
 'Can',
 'Chris',
 'Come',
 'Congrats',
 'Cool',
 'D',
 'DM',
 'DO',
 'Damn',
 'Day',
 'Did',
 'Do',
 'Enjoy',
 'FF',
 'Follow',
 'FollowFriday',
 'For',
 'Friday',
 'Get',
 'Glad',
 'Go',
 'God',
 'Good',
 'Got',
 'Great',
 'Had',
 'Haha',
 'Happy',
 'Have',
 'He',
 'Hello',
 'Hey',
 'Hi',
 'Hope',
 'How',
 'I',
 'IS',
 'IT',
 'If',
 'Im',
 'In',
 'Is',
 'It',
 'Its',
 'July',
 'June',
 'Just',
 'Keep',
 'LA',
 'LMAO',
 'LOL',
 'LOVE',
 'Let',
 'Like',
 'Lol',
 'London',
 'Love',
 'ME',


In [25]:
def lemmatize_tokenize(tweets):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in word_tokenize(tweets)]

In [26]:
class TextPrepare_tweets(BaseEstimator,TransformerMixin):
    def __init__(self, use_mention=False):
        self.use_mention = use_mention
    
    def fit(self, tweets, y=None):
        return self
    
    def transform(self, tweets, y=None):
        # We can choose between keeping the mentions
        # or deleting them
        if self.use_mention:
            tweets = tweets.str.replace(r"@[a-zA-Z0-9_]* ", " @tags ")
        else:
            tweets = tweets.str.replace(r"@[a-zA-Z0-9_]* ", "")
            
        #Remove punctuations:
        tweets = tweets.str.strip('\'"?!,.():;')
        # Remove repetative characters:
        tweets = tweets.str.replace(r"(.)\1+", r"\1\1")
        #Remove urls and replace with " "
        tweets = tweets.str.replace(r"https?://\S*", "")
        #Remove @username and replace with " "
        tweest = tweets.str.replace(r"@[^\s]+", "")
        #Remove multiple white spaces and replace with a single 
        tweets =tweets.str.replace(r"[\s]+", "")
        #Remove "Hashtag(#word)" and replace with  "word"
        tweets =tweets.str.replace(r"#([^\s]+)", r"\1")
        #Remove 2+ dots and replace with space
        tweets =tweets.str.replace(r"\.{2,}", "")
        #Remove special characters except letters and numbers and replace with space
        tweets =tweets.str.replace("[^ a-zA-Z0-9]", "")
        #Remove numbers with space [Numbers doesn't help to determine positive or negative emotions]
        tweets =tweets.str.replace("[0-9]", "")
        #Remove retweets and CC [Retweets always begins with RT]
        tweets =tweets.str.replace("RT|cc", "")
        # mark emoticons as happy or sad
        tweets = tweets.str.replace(POSITIVE_EMO, " positiveemoticons ")
        tweets = tweets.str.replace(NEGATIVE_EMO, " negativeemoticons ")
        tweets = tweets.str.lower()
        return tweets

#Prepare the data:
Bag of Words: BOW is a method to extract features from text documents. These features can be used for training machine learning algorithms. It creates a vocabulary of all the unique words occurring in all the documents in the training set.
TF-IDF (Term Frequency - Inverse Document Frequency)
->scikit-learn provides utilities for the most common ways to extract numerical features from text content, namely:

1. tokenizing strings and giving an integer id for each possible token, for instance by using white-spaces and punctuation as token separators.

2. counting the occurrences of tokens in each document.

3. normalizing and weighting with diminishing importance tokens that occur in the majority of samples / documents.

In [9]:
sentiments = train_data['Sentiment']
tweets = train_data['SentimentText']

In [10]:
vectorizer = TfidfVectorizer(tokenizer=lemmatize_tokenize, ngram_range=(1,2))
pipeline = Pipeline([
    ('text_pre_processing', TextPrepare_tweets(use_mention=True)),
    ('vectorizer', vectorizer)])
learn_data, test_data, sentiments_learning, sentiments_test = train_test_split(tweets, sentiments, test_size=0.3)
learning_data = pipeline.fit_transform(learn_data)

MODEL SELECTION:
Naive Bias has been proven good for Text Based Problems.
Steps:
A good way to choose a model is to try different models, evaluate them using cross validation, then choose the best one which will be later tested against our test data (test_data).

In [11]:
lr = LogisticRegression()
bnb = BernoulliNB()
mnb = MultinomialNB()

models = {
    'logitic regression': lr,
    'bernoulliNB': bnb,
    'multinomialNB': mnb,
}

for model in models.keys():
    scores = cross_val_score(models[model], learning_data, sentiments_learning, scoring="f1", cv=10)
    print("===", model, "===")
    print("scores = ", scores)
    print("mean = ", scores.mean())
    print("variance = ", scores.var())
    models[model].fit(learning_data, sentiments_learning)
    print("score on the learning data (accuracy) = ", accuracy_score(models[model].predict(learning_data), sentiments_learning))
    print("")
    

/home/lipsa/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


=== logitic regression ===
scores =  [0.72563442 0.72633707 0.72630223 0.72549922 0.72482113 0.72351753
 0.72616857 0.72595047 0.72641943 0.7261861 ]
mean =  0.7256836181258715
variance =  7.382207398542058e-07
score on the learning data (accuracy) =  0.9959709681106412

=== bernoulliNB ===
scores =  [0.72608656 0.72605382 0.72620252 0.725802   0.72428964 0.72373612
 0.72595181 0.72596817 0.72638646 0.72598584]
mean =  0.7256462951794407
variance =  7.042886017694436e-07
score on the learning data (accuracy) =  0.9956566464738827

=== multinomialNB ===
scores =  [0.72563442 0.72633707 0.72630223 0.72549922 0.72482113 0.72351753
 0.72616857 0.72595047 0.72641943 0.7261861 ]
mean =  0.7256836181258715
variance =  7.382207398542058e-07
score on the learning data (accuracy) =  0.9959709681106412



Fine tune the model


In [12]:
grid_search_pipeline = Pipeline([
    ('text_pre_processing', TextPrepare_tweets()),
    ('vectorizer', TfidfVectorizer()),
    ('model', MultinomialNB()),
])

params = [
    {
        'text_pre_processing__use_mention': [True, False],
        'vectorizer__max_features': [1000, 2000, 5000, 10000, 20000, None],
        'vectorizer__ngram_range': [(1,1), (1,2)],
    },
]
grid_search = GridSearchCV(grid_search_pipeline, params, cv=5, scoring='f1')
grid_search.fit(learn_data, sentiments_learning)
print(grid_search.best_params_)

{'text_pre_processing__use_mention': False, 'vectorizer__max_features': None, 'vectorizer__ngram_range': (1, 1)}


TEST

In [13]:
mnb.fit(learning_data, sentiments_learning)
testing_data = pipeline.transform(test_data)
mnb.score(testing_data, sentiments_test)

0.5715238190485715

In [14]:
# Predecting on the test.csv
test_data = pd.read_csv("Desktop/AI Docs/project/test.csv", encoding='ISO-8859-1')
test_learning = pipeline.transform(test_data.SentimentText)
test = pd.DataFrame(test_data.ItemID, columns=("ItemID", "Sentiment"))
test["Sentiment"] = mnb.predict(test_learning)
print(test)

        ItemID  Sentiment
0            1          0
1            2          1
2            3          1
3            4          0
4            5          0
5            6          0
6            7          1
7            8          1
8            9          1
9           10          1
10          11          1
11          12          1
12          13          0
13          14          0
14          15          0
15          16          0
16          17          0
17          18          1
18          19          0
19          20          0
20          21          0
21          22          1
22          23          1
23          24          0
24          25          1
25          26          0
26          27          0
27          28          0
28          29          1
29          30          0
...        ...        ...
299959  299971          1
299960  299972          1
299961  299973          1
299962  299974          1
299963  299975          1
299964  299976          1
299965  2999

TEST THE TWEET

In [15]:
model = MultinomialNB()
model.fit(learning_data, sentiments_learning)
tweet = pd.Series([input(),])
tweet = pipeline.transform(tweet)
proba = model.predict_proba(tweet)[0]
print("The probability that this tweet is negative is:", proba[0])
print("The probability that this tweet is positive is:", proba[1])

Be Happy
The probability that this tweet is negative is: 0.3347639730254885
The probability that this tweet is positive is: 0.6652360269745115
